# ทดสอบการเข้ารหัสคอลัมน์ใน Postgres ด้วย `pgcrypto`

## เตรียมสภาพแวดล้อม
- รัน `docker-compose up -d` เพื่อเริ่มฐานข้อมูล (มีการเปิดใช้ `pgcrypto` อัตโนมัติ)
- แนะนำให้ใช้ virtual environment และติดตั้งไลบรารีใน `requirements.txt`

In [ ]:
# ติดตั้งไลบรารีที่จำเป็น (ถ้ายังไม่มี)
# หมายเหตุ: สามารถข้ามเซลล์นี้ได้ถ้าติดตั้งไว้แล้วใน venv
# !pip install -q "psycopg[binary]>=3.1" pandas

In [1]:
# การตั้งค่าการเชื่อมต่อฐานข้อมูล
import os
import psycopg
import pandas as pd

HOST = os.getenv('PG_HOST', 'localhost')
PORT = int(os.getenv('PG_PORT', '5432'))
DB   = os.getenv('PG_DB', 'encdb')
USER = os.getenv('PG_USER', 'encuser')
PWD  = os.getenv('PG_PASSWORD', 'encpass')
# คีย์สำหรับการเข้ารหัส/ถอดรหัสแบบ symmetric (เพื่อการทดสอบเท่านั้น)
PASSPHRASE = os.getenv('PG_PASSPHRASE', 'my-strong-demo-passphrase')

def get_conn():
    return psycopg.connect(host=HOST, port=PORT, dbname=DB, user=USER, password=PWD)

conn = get_conn()
conn.autocommit = True
cur = conn.cursor()
print('เชื่อมต่อฐานข้อมูลสำเร็จ')

เชื่อมต่อฐานข้อมูลสำเร็จ


## 1) เริ่มต้นฐานข้อมูลและสร้างตารางตัวอย่าง

In [2]:
# สร้างส่วนขยาย pgcrypto (ถ้ายังไม่ได้เปิดใช้จากสคริปต์เริ่มต้น)
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")

# ลบตารางเดิม (ถ้ามี) เพื่อทดสอบซ้ำได้
cur.execute("DROP TABLE IF EXISTS customer_secret;")

# สร้างตารางที่มีคอลัมน์เข้ารหัสแบบ bytea โดยใช้ pgp_sym_encrypt
cur.execute(
    """
    CREATE TABLE customer_secret (
        id SERIAL PRIMARY KEY,
        full_name TEXT NOT NULL,
        national_id_encrypted BYTEA NOT NULL -- เก็บข้อมูลหลังเข้ารหัส
    );
    """
)
print('สร้างตาราง customer_secret สำเร็จ')

สร้างตาราง customer_secret สำเร็จ


## 2) เพิ่มข้อมูลโดยเข้ารหัส (Insert with Encrypt)

In [4]:
# ตัวอย่างข้อมูลที่ต้องการปกป้อง เช่น เลขบัตรประชาชน
samples = [
    ('สมชาย ใจดี', '1101700200011'),
    ('สมหญิง แข็งแรง', '1101700200022'),
    ('สายฝน สายชล', '1101700200033'),
]

# ใช้ pgp_sym_encrypt เพื่อเข้ารหัสคอลัมน์ national_id_encrypted
for full_name, nid in samples:
    cur.execute("INSERT INTO customer_secret (full_name, national_id_encrypted) VALUES (%s, pgp_sym_encrypt(%s, %s));",
        (full_name, nid, PASSPHRASE)
    )
print('เพิ่มข้อมูลแบบเข้ารหัสสำเร็จ')

เพิ่มข้อมูลแบบเข้ารหัสสำเร็จ


## 2.1) ดูข้อมูลที่เข้ารหัสแล้วในฐานข้อมูล (View Encrypted Data)

In [5]:
# ดูข้อมูลที่เข้ารหัสแล้วตามที่เก็บในฐานข้อมูล
# จะเห็นว่าคอลัมน์ national_id_encrypted เป็นข้อมูลที่เข้ารหัสแล้ว (bytea)
cur.execute("SELECT id, full_name, national_id_encrypted FROM customer_secret ORDER BY id;")
encrypted_rows = cur.fetchall()
encrypted_df = pd.DataFrame(encrypted_rows, columns=['id', 'full_name', 'national_id_encrypted'])
print("ข้อมูลที่เข้ารหัสในฐานข้อมูล:")
encrypted_df

ข้อมูลที่เข้ารหัสในฐานข้อมูล:


,id,full_name,national_id_encrypted
0,1,สมชาย ใจดี,"b'\xc3\r\x04\x07\x03\x02""z\xc5\x81\x93\x00\xcb..."
1,2,สมหญิง แข็งแรง,b'\xc3\r\x04\x07\x03\x02\xf3\x80\x86#HWr\x94~\...
2,3,สายฝน สายชล,b'\xc3\r\x04\x07\x03\x02\xad\x8b\xf5\xbb\x02\x...


## 3) ค้นหาข้อมูลโดยถอดรหัส (Select with Decrypt)

In [6]:
# ใช้ pgp_sym_decrypt เพื่อถอดรหัสคอลัมน์ national_id_encrypted ขณะคิวรี
cur.execute("SELECT id, full_name, pgp_sym_decrypt(national_id_encrypted, %s) AS national_id \
       FROM customer_secret \
       ORDER BY id;",
    (PASSPHRASE,)
)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['id', 'full_name', 'national_id'])
df

,id,full_name,national_id
0,1,สมชาย ใจดี,1101700200011
1,2,สมหญิง แข็งแรง,1101700200022
2,3,สายฝน สายชล,1101700200033


### หมายเหตุด้านความปลอดภัย
- ตัวอย่างนี้เพื่อการสาธิต: เก็บ `passphrase` ในตัวแปรโน้ตบุ๊กเพื่อความง่าย
- ในระบบจริงควรดึงคีย์จาก secret manager/ENV และกำหนดสิทธิ์อย่างเหมาะสม
- `pgp_sym_encrypt/pgp_sym_decrypt` คืนค่า/ใช้ข้อมูลแบบ `bytea` ในตาราง
- หลีกเลี่ยงการเข้ารหัสข้อมูลที่ต้องใช้ค้นหาแบบเท่ากัน/ช่วง (index ใช้ไม่ได้กับข้อมูลที่เข้ารหัสแบบนี้)

In [7]:
# ปิดการเชื่อมต่อเมื่อทดสอบเสร็จ
cur.close()
conn.close()
print('ปิดการเชื่อมต่อฐานข้อมูลแล้ว')

ปิดการเชื่อมต่อฐานข้อมูลแล้ว
